In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 58.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 109.1 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

## 导入数据集

将数据集使用 Pandas 导入

In [7]:
df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)

为缩减训练时间,只采用前2000个数据
你可以选择更多的数据进行验证

In [8]:
batch_1 = df[:2000]

查看原始数据-2000条数据-集中有0和1标签的分布

In [9]:
batch_1[1].value_counts()

1    1041
0     959
Name: 1, dtype: int64

## 加载预训练好的 Bert 模型

In [10]:
# 对于 DistilBERT:
# 首先设定要使用的模型类别、分词器类别以及预训练模型的权重名称
# 这里选择DistilBertModel（模型类别），DistilBertTokenizer（分词器类别），以及'distilbert-base-uncased'（预训练模型的权重名称）
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## 想要使用BERT就取消以下行的注释：
# 这一行代码是为了如果你更喜欢使用原始的BERT模型而设定的，如果你取消这行代码的注释，那么你将加载BERT模型而不是DistilBERT模型
# 记得注释上面那一行DistilbertModel的代码
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# 加载预训练模型/分词器
# 从预训练的模型权重中加载分词器和模型。，"from_pretrained"函数可以直接加载预训练的模型和分词器
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Model #1: 经典数据预处理流程

### Tokenization
第一步将句子转化为words，在转化为subwords，确保能满足Bert模型的输入要求

In [11]:
# 使用预训练的分词器对数据进行分词
# batch_1[0]表示我们要处理的数据
# "apply"函数用于对数据中的每个元素进行操作，这里的操作是一个lambda函数 aka 匿名函数
# 在这个匿名函数中，调用了"tokenizer.encode"函数，对每个元素（这里是每个文本）进行分词。
# "add_special_tokens=True"表示在分词的过程中添加特殊的标记，包括开始和结束的标记等，这些不能少，Bert模型 requirements
tokenized = batch_1[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))



### Padding

文本预处理的经典步骤之-Padding 填充，确保输入的token长度是相同的

In [12]:
# 寻找最大的序列长度
# 首先要找出所有分词后的句子中最长的句子的长度,创建一个变量max_len，并将其初始值设为0，然后遍历所有的句子，如果一个句子的长度大于max_len，更新max_len的值
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

# 对序列进行填充
# 然后对所有的句子进行填充，使它们的长度都等于max_len。填充的方式是在句子的后面添加0，直到句子的长度等于max_len
# 用np.array来存储填充后的句子。每一个句子都是一个数组，所有的句子构成了一个二维数组
padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])


Our dataset is now in the `padded` variable, we can view its dimensions below:

In [ ]:
np.array(padded).shape

(2000, 59)

### Masking

对于基于Encoder 构建的 Bert，需要对 Attention 进行mask处理，确保Padding 出来的0不会对 Attention 过程产生side effects

In [13]:
# 创建注意力掩码
# 注意力掩码用于指示模型哪些位置的输入是真正的词语，哪些位置是填充的部分
# 创建一个与padded数组形状相同的attention_mask数组，该数组中，padded中每个非零位置（即实际词语的位置）对应的值为1，零位置（即填充的位置）对应的值为0
# np.where函数的作用是根据条件选择元素。条件是padded不等于0，如果条件为真（即位置上的元素不为0，是实际的词语），则选择1，否则选择0
attention_mask = np.where(padded != 0, 1, 0)

attention_mask.shape


(2000, 59)

将与处理好的数据送给模型进行处理，得到模型的输出结果

In [14]:
# 将输入数据和注意力掩码转化为PyTorch张量
# PyTorch的模型需要使用PyTorch的张量作为输入，所以需要将padded和attention_mask从NumPy数组转化为PyTorch张量 - required
# 使用torch.tensor函数完成转化
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

# 使用模型进行前向传播
# 使用with torch.no_grad()语句块来锁定梯度计算
# 在进行前向传播时，并不需要计算梯度，关闭梯度计算可以节省内存，提高计算速度
# 然后，将input_ids和attention_mask作为输入，通过模型进行前向传播，得到输出last_hidden_states
# attention_mask=attention_mask将attention_mask作为名为"attention_mask"的参数传递给模型
# 这段代码会基于你的GPU的能力变化训练时间，如果训练时间过长，那就把前面的数据集再改小一点，2000变1000， 1000 变 500
# 我大概跑了4分钟，希望能作为一个基准
with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)



因为在Bert模型中，输入句子转换后的数据的第一个位置是 [cls]，cls 代表的就是整个句子的 Sentence Encoding 或者说是 Sentence presentation，而作为下游的判别任务的依据就是这个 cls token

In [17]:
# 提取特征
# 从last_hidden_states（模型的输出，包含了每个输入词语的隐藏状态）中提取特征
# last_hidden_states[0]取出了隐藏状态，因为模型的输出是一个元组，其第一个元素是隐藏状态
# [:,0,:]表示取出每个句子的第一个词语（BERT模型中的CLS标记）的所有隐藏状态
# 在BERT模型中，第一个词语的隐藏状态被用作句子的表示，所以我们通常会使用它作为特征
# .numpy()将隐藏状态从PyTorch张量转化为NumPy数组，因为我们接下来可能会使用NumPy或者其他库来处理这些特征
# 因为后续会将这个特征送给一个NN来处理
features = last_hidden_states[0][:,0,:].numpy()


Batch中的[1] 代表真实的标签，0或1

In [16]:
labels = batch_1[1]

## Model #2: Train/Test Split

将数据切分成训练集和测试集，直接使用API切分了，省事

In [18]:
# 划分训练集和测试集
# 使用sklearn库的train_test_split函数将数据划分为训练集和测试集
# "features"是刚刚计算的到的特征，"labels"是训练集的标签
# 函数返回的四个值分别是训练集特征、测试集特征、训练集标签和测试集标签
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)


训练最后的逻辑回归模型的参数
虽然Bert是预训练过的，但是在做下游任务时的模型是没有训练的

In [19]:
# 创建逻辑回归模型
# 使用sklearn库的LogisticRegression类创建了一个逻辑回归模型
lr_clf = LogisticRegression()

# 训练逻辑回归模型
# 使用fit方法来训练模型
# fit方法需要两个参数：训练集的特征和训练集的标签，在上一步得到了
lr_clf.fit(train_features, train_labels)


LogisticRegression()

计算训练效果

In [20]:
# 评估模型
# 使用score方法来评估模型在测试集上的表现
# score方法需要两个参数：测试集的特征和测试集的标签
# score方法会返回模型在测试集上的准确率，即正确预测的样本数占总样本数的比例, 体现模型在真实场景中的能力
lr_clf.score(test_features, test_labels)


0.83

只得到自己的得分其实很不合理，考试都得和别人比
这里使用一个 dummy classifier （模拟分类器/或者叫傻瓜分类器）来进行分类，比较得分
Dummy 是不会进行任何学习的，只是简单地使用一些规则（如预测所有样本都属于最常见的类别）来进行预测，比如使用众数进行分类

In [21]:
# 导入DummyClassifier
# DummyClassifier是一种简单的分类器，它不进行任何学习，只是简单地使用一些规则（如预测所有样本都属于最常见的类别）来进行预测
from sklearn.dummy import DummyClassifier

# 创建Dummy分类器
clf = DummyClassifier()

# 使用交叉验证来评估模型的性能
# cross_val_score函数会将数据集划分为k个子集，然后进行k次训练和测试
# 每次，都会选择一个子集作为测试集，其他的子集作为训练集
# cross_val_score函数返回的是每次测试的分数（在这里是准确率）
# 计算这些分数的平均值和标准差来评估模型的性能
scores = cross_val_score(clf, train_features, train_labels)

# Dummy分类器的平均分数和95%置信区间（即平均分数±两倍的标准差）的方式来进行比较
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))


Dummy classifier score: 0.511 (+/- 0.00)


0.8 大于 0.5  只能说这个得分比傻子好点，
- 事实上我们如果增加数据集的大小，
- 将固定参数的方式改为  fine tunning 的方式进行会更好
- 但那也会消耗更多的时间进行训练

在本demo中不进行展示了